In [51]:
from sshtunnel import SSHTunnelForwarder
import pymongo
import pprint
import pandas as pd
from datetime import datetime, timezone
import os
import requests
from binance.client import Client


MONGO_HOST = "35.205.115.90"
MONGO_DB = "cryptocurrency"

server = SSHTunnelForwarder(
    MONGO_HOST,
#     ssh_username="YOURUSERNAME",
    ssh_pkey='~/.ssh/id_rsa', # your ssh filepath
#     ssh_private_key_password="YOURPASSWORD",
    remote_bind_address=('127.0.0.1', 27017)
)

server.start()

client = pymongo.MongoClient('127.0.01', server.local_bind_port)
db = client[MONGO_DB]
collection = db['cbpro']
pprint.pprint(db.collection_names())
pprint.pprint(collection.find({'date': '2020-09-12'}))
pprint.pprint(collection.count())
server.stop()

['okex',
 'binance',
 'deribit_orderbooks',
 'bitstamp',
 'deribit_transactions',
 'poloniex',
 'cbpro',
 'hitbtc',
 'bitfinex']
118607177


In [22]:
doc = collection.find({'date': '2020-09-12'})
pprint.pprint(doc)

In [30]:
db.cbpro.find_one()

{'_id': ObjectId('5ca4871afe7ef136997c586e'),
 'product_id': 'BTC-USD',
 'sequence': 8186993302,
 'taker_order_id': 'ab063085-3a50-4ba2-abde-3723db9845a9',
 'price': '4975.46000000',
 'trade_id': 61348510,
 'time': '2019-04-03T10:12:41.387000Z',
 'maker_order_id': 'e42d1460-48a5-4a22-b2a4-a91afcd9ac7a',
 'type': 'last_match',
 'side': 'sell',
 'size': '0.00470168'}

## Coinbase

SIDE
The trade side indicates the maker order side. The maker order is the order that was open on the order book. buy side indicates a down-tick because the maker was a buy order and their order was removed. Conversely, sell side indicates an up-tick.

https://docs.pro.coinbase.com/#get-historic-rates


GET /products/<product-id>/candles

start	Start time in ISO 8601
end	End time in ISO 8601
granularity	Desired timeslice in seconds

for market share
https://www.theblockcrypto.com/data/crypto-markets/spot/cryptocurrency-exchange-volume-market-share-monthly
1st: Binance
2nd: Coinbase
3rd: Kraken
4th: LMAX Digital

In [31]:
url = "https://api.pro.coinbase.com"
product_id = 'BTC-USD'
start = '?start=2021-02-16'
end = '?end=2021-02-17'
print(url+'/'+product_id+'/candles'+start+'&'+end)

https://api.pro.coinbase.com/BTC-USD/candles?start=2021-02-16&?end=2021-02-17


In [62]:
import cbpro

public_client = cbpro.PublicClient()
data =public_client.get_product_historic_rates('BTC-USD',
                                               start='2021-02-10', 
                                               end='2021-02-18',
                                               granularity=3600) #{60, 300, 900, 3600, 21600, 86400}

In [66]:
data = pd.DataFrame(data)
data

,0,1,2,3,4,5
0,1613577600,50782.42,51453.99,51172.34,51158.64,962.410877
1,1613574000,50992.86,51646.49,51240.66,51172.34,1384.589463
2,1613570400,50720.80,51250.00,51158.76,51240.65,1122.936187
3,1613566800,50500.00,51217.43,51049.50,51151.20,1196.292536
4,1613563200,50795.37,51544.42,51270.00,51043.44,1102.862586
5,1613559600,51131.19,51700.00,51214.11,51277.14,691.591415
6,1613556000,51070.00,51649.85,51483.96,51204.26,524.344202
7,1613552400,51070.34,51717.88,51071.92,51484.40,892.553369
8,1613548800,50610.00,51288.00,51114.65,51071.92,900.896726
9,1613545200,50280.52,51248.00,50588.04,51114.65,1667.360824


In [64]:
for i in range(10):
    print(datetime.fromtimestamp(datetimes[i]).month,
          datetime.fromtimestamp(datetimes[i]).day,
          datetime.fromtimestamp(datetimes[i]).hour, 
          datetime.fromtimestamp(datetimes[i]).minute)

NameError: name 'datetime' is not defined

In [65]:
api_key = os.environ.get('binance_api')
api_secret = os.environ.get('binance_secret')
client = Client(api_key, api_secret)

In [5]:
depth = client.get_order_book(symbol='BNBBTC')

In [39]:
# client.get_historical_klines_generator()
import pandas as pd
import time
t = pd.Timestamp('2021-02-10 00:00:00', 'ms')
t = time.mktime(t.timetuple())
t = str(int(t))

TypeError: Tuple or struct_time argument required

In [56]:
t = pd.Timestamp(1613433600000*1000000)
pd.to_datetime(t)

Timestamp('2021-02-16 00:00:00')

In [52]:
start_day = "2021-02-15T00:00+00:00"
end_day = "2021-02-16T00:00+00:00"

In [57]:
binance_price = []
for kline in client.get_historical_klines_generator("BTCUSDT",
                                                    Client.KLINE_INTERVAL_1HOUR,
                                                    start_day, end_day):
    binance_price.append(kline)

In [59]:
bp = pd.DataFrame(binance_price)
bp.columns = ['OpenTime', 'Open', 'High', 'Low', 'Close', 'Volume', 'CloseTime', 'Quote asset volume', 'N trades', 'taker1','taker2', 'ignore']

In [61]:
bp

,OpenTime,Open,High,Low,Close,Volume,CloseTime,Quote asset volume,N trades,taker1,taker2,ignore
0,1613347200000,48580.47000000,49010.92000000,48428.00000000,48763.62000000,2016.44349500,1613350799999,98237902.75557831,81903,970.97984000,47312216.92817248,0
1,1613350800000,48763.79000000,48816.62000000,47700.00000000,48011.44000000,4889.30019600,1613354399999,235426059.12818572,137267,2002.07846900,96414045.04540308,0
2,1613354400000,48011.45000000,48213.83000000,45570.79000000,46874.02000000,11217.85149100,1613357999999,524402408.59355430,293802,4766.42986400,222833708.12581665,0
3,1613358000000,46874.01000000,47269.09000000,46540.67000000,47120.30000000,4188.45200300,1613361599999,196816109.46067822,130430,1999.77789900,93986906.83760872,0
4,1613361600000,47120.31000000,47136.02000000,46174.78000000,46866.57000000,4963.73452400,1613365199999,231640212.72911710,117265,2555.71418300,119198071.11229076,0
5,1613365200000,46866.56000000,47000.00000000,46400.00000000,46722.49000000,2939.43039800,1613368799999,137310368.09635524,80552,1310.16549500,61211676.80102441,0
6,1613368800000,46722.48000000,47140.00000000,46683.02000000,46887.36000000,2622.53921200,1613372399999,123158432.47353344,89824,1244.34040700,58439504.94655495,0
7,1613372400000,46887.36000000,47441.02000000,46831.61000000,47267.06000000,2861.55980200,1613375999999,135181501.07298136,82240,1418.34999000,66996900.89167774,0
8,1613376000000,47270.50000000,47772.00000000,47112.38000000,47599.99000000,2821.99134500,1613379599999,133918267.12457979,77978,1474.81929500,69985910.06565023,0
9,1613379600000,47600.00000000,47724.64000000,47201.27000000,47505.66000000,3001.82932500,1613383199999,142576622.11924442,70082,1336.05093400,63455178.90359955,0
